## 1. Experiment Tracking:
- Scenario 1: MLflow on localhost
- Both the backend and artifact store share a directory on the local filesystem—./mlruns
- In this simple scenario, the MLflow client uses the following interfaces to record MLflow entities and artifacts:
- An instance of a LocalArtifactRepository (to store artifacts)
- An instance of a FileStore (to save MLflow entities)



In [1]:
import logging

logger = logging.getLogger("mlflow")

# Set log level to debugging
logger.setLevel(logging.DEBUG)

In [2]:
import sys

In [3]:
sys.executable

'/media/indra-inc/ML_Doc/mflow_projects/mlflow-env/bin/python'

### Model training Life Cycle:

In [4]:
import pandas as pd
import numpy as np

In [5]:
## Reading the dataset
def load_data(path):
    data = pd.read_csv(path)
    return data

In [6]:
data = load_data("banking.csv")

In [7]:
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,44,blue-collar,married,basic.4y,unknown,yes,no,cellular,aug,thu,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,53,technician,married,unknown,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,management,single,university.degree,no,yes,no,cellular,jun,thu,...,3,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,services,married,high.school,no,no,no,cellular,apr,fri,...,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,retired,married,basic.4y,no,yes,no,cellular,aug,fri,...,1,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,1


In [8]:
data.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx',
       'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')

In [9]:
## Little Data Cleaning
def data_cleaning(data):
    print(f"Missing Values Before: {data.isna().sum()}")

    # Droping Missing values
    data = data.dropna()

    print(f"Missing Values After: {data.isna().sum()}")
    return data

In [10]:
## EDA or Data Preprocessing
def preprocessing(data):
    data['education']=np.where(data['education'] =='basic.9y', 'Basic', data['education'])
    data['education']=np.where(data['education'] =='basic.6y', 'Basic', data['education'])
    data['education']=np.where(data['education'] =='basic.4y', 'Basic', data['education'])
    
    cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
    for var in cat_vars:
        cat_list='var'+'_'+var
        cat_list = pd.get_dummies(data[var], prefix=var)
        data1=data.join(cat_list)
        data=data1

    cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
    data_vars=data.columns.values.tolist()
    to_keep=[i for i in data_vars if i not in cat_vars]
    
    final_data=data[to_keep]
    
    
    final_data.columns = final_data.columns.str.replace('.','_')
    final_data.columns = final_data.columns.str.replace(' ','_')
    return final_data

In [11]:
def train_test_split(final_data):
    from sklearn.model_selection import train_test_split
    X = final_data.loc[:, final_data.columns != 'y']
    y = final_data.loc[:, final_data.columns == 'y']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify = y, random_state=47)
    return X_train, X_test, y_train, y_test

In [12]:
def over_sampling_target_class(X_train, y_train):
    ### Over-sampling using SMOTE 
    from imblearn.over_sampling import SMOTE
    os = SMOTE(random_state=0)

    columns = X_train.columns
    os_data_X,os_data_y=os.fit_resample(X_train, y_train)

    os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
    os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])
    # we can Check the numbers of our data
    print("length of oversampled data is ",len(os_data_X))
    print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['y']==0]))
    print("Number of subscription",len(os_data_y[os_data_y['y']==1]))
    print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==0])/len(os_data_X))
    print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==1])/len(os_data_X))
    
    X_train = os_data_X
    y_train = os_data_y['y']
 
    return X_train, y_train

In [13]:
def training_basic_classifier(X_train,y_train):
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(n_estimators=101)
    model.fit(X_train, y_train)
    
    return model

In [14]:
def predict_on_test_data(model,X_test):
    y_pred = model.predict(X_test)
    return y_pred

In [15]:
def predict_prob_on_test_data(model,X_test):
    y_pred = model.predict_proba(X_test)
    return y_pred

In [16]:
def get_metrics(y_true, y_pred, y_pred_prob):
    from sklearn.metrics import accuracy_score,precision_score,recall_score,log_loss
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    entropy = log_loss(y_true, y_pred_prob)
    return {'accuracy': round(acc, 2), 'precision': round(prec, 2), 'recall': round(recall, 2), 'entropy': round(entropy, 2)}

In [19]:
def create_confusion_matrix_plot(y_true, y_pred):
    import matplotlib.pyplot as plt
    from sklearn.metrics import confusion_matrix
    mat = confusion_matrix(y_true, y_pred)
    print(mat)

## Run the application Stage-1:

In [20]:
cleaned_data = data_cleaning(data)

Missing Values Before: age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp_var_rate      0
cons_price_idx    0
cons_conf_idx     0
euribor3m         0
nr_employed       0
y                 0
dtype: int64
Missing Values After: age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp_var_rate      0
cons_price_idx    0
cons_conf_idx     0
euribor3m         0
nr_employed       0
y                 0
dtype: int64


In [21]:
final_data = preprocessing(cleaned_data)        # Preprocessed data

In [22]:
X_train, X_test, y_train, y_test = train_test_split(final_data)             # Data Splitting

In [23]:
y_test

,y
11852,0
25082,0
14380,0
24906,0
18867,0
...,...
16110,0
8906,0
19766,0
2163,0


In [24]:
model_1 = training_basic_classifier(X_train,y_train)        ## RandomForest Classifier without any hyperparameter tuning

/media/indra-inc/ML_Doc/mflow_projects/mlflow-env/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [25]:
y_pred = predict_on_test_data(model_1,X_test)           # Prediction data
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [26]:
y_pred_prob = predict_prob_on_test_data(model_1,X_test)         # model.predict_proba(X_test)
y_pred_prob

array([[1.        , 0.        ],
       [0.94059406, 0.05940594],
       [0.97029703, 0.02970297],
       ...,
       [1.        , 0.        ],
       [0.81188119, 0.18811881],
       [1.        , 0.        ]])

In [27]:
run_metrics = get_metrics(y_test, y_pred, y_pred_prob)          # Getting metrics
run_metrics

{'accuracy': 0.91, 'precision': 0.66, 'recall': 0.46, 'entropy': 0.21}

In [28]:
create_confusion_matrix_plot(y_test,y_pred)

[[10642   323]
 [  758   634]]


# MLflow workflow 

In [29]:
import mlflow

In [30]:
experiment_name = "Banking Data Classification"     ## Experiment name for the banking data
run_name = "my_first_run"
# run_metrics = get_metrics(y_test, y_pred, y_pred_prob)
model_name = "Random_Forest_without_Optimization"

## Function to create an experiment in MLFlow and log parameters, metrics and artifacts files
- This function will be usefull for both the cases i.e. before and after hyperparameter tuning

In [31]:
def create_experiment(experiment_name,run_name, model_name, run_metrics,model, run_params=None):
    # import mlflow
    # mlflow.set_tracking_uri("http://localhost:5000") #uncomment this line if you want to use any database like sqlite as backend storage for model
    experiment = mlflow.set_experiment(experiment_name)
    # Get Experiment Details
    print("Experiment_id: {}".format(experiment.experiment_id))
    
    with mlflow.start_run():
        run = mlflow.active_run()
        if not run_params == None:
            for param in run_params:
                mlflow.log_param(param, run_params[param])
            
        for metric in run_metrics:
            mlflow.log_metric(metric, run_metrics[metric])
        
        mlflow.sklearn.log_model(model, "model")
        
        mlflow.set_tag('Developers', "Indra-Inc")
        mlflow.set_tag("Model", model_name)
        # mlflow.set_tags({"tag2":"Randomized Search CV", "tag3":"Production"})
            
    print(f"Run {run_name} with Run ID {run.info.run_id} is logged to Experiment {experiment_name} having status: {run.info.status}")

## Create and Run model without Hyperparameter Tuning

In [32]:
create_experiment(experiment_name,run_name,model_name,run_metrics,model_1)

2023/09/11 03:25:18 INFO mlflow.tracking.fluent: Experiment with name 'Banking Data Classification' does not exist. Creating a new experiment.


Experiment_id: 484267182419968048
Run my_first_run with Run ID 90969c749fc2403281d7140ad7e69453 is logged to Experiment Banking Data Classification having status: RUNNING


## Creating another Experiment after tuning hyperparameters and log the best set of parameters for which model gives the optimal performance

#### Stage-2: Model with Hyperparameter

In [33]:
def hyper_parameter_tuning(X_train, y_train):
    # define random parameters grid
    n_estimators = [5,21,51,101] # number of trees in the random forest
    max_features = ['auto', 'sqrt'] # number of features in consideration at every split
    max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
    min_samples_split = [2, 6, 10] # minimum sample number to split a node
    min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
    bootstrap = [True, False] # method used to sample data points

    random_grid = {'n_estimators': n_estimators,
                    'max_features': max_features,
                    'max_depth': max_depth,
                    'min_samples_split': min_samples_split,
                    'min_samples_leaf': min_samples_leaf,
                    'bootstrap': bootstrap
                  }
    
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier()
    model_tuning = RandomizedSearchCV(estimator = classifier, param_distributions = random_grid,
                   n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)
    model_tuning.fit(X_train, y_train)

    print ('Random grid: ', random_grid, '\n')
    # print the best parameters
    print ('Best Parameters: ', model_tuning.best_params_, ' \n')

    best_params = model_tuning.best_params_
    
    n_estimators = best_params['n_estimators']
    min_samples_split = best_params['min_samples_split']
    min_samples_leaf = best_params['min_samples_leaf']
    max_features = best_params['max_features']
    max_depth = best_params['max_depth']
    bootstrap = best_params['bootstrap']
    
    model_tuned = RandomForestClassifier(n_estimators = n_estimators, min_samples_split = min_samples_split,
                                         min_samples_leaf= min_samples_leaf, max_features = max_features,
                                         max_depth= max_depth, bootstrap=bootstrap) 
    model_tuned.fit( X_train, y_train)
    return model_tuned,best_params

In [34]:
run_name_tuned="Random_Search_CV_Tuned_Model"
model_name = "Random_Forest_with_Random_Search_Optimization"

In [35]:
model_tuned,best_params = hyper_parameter_tuning(X_train, y_train)
run_params = best_params

y_pred = predict_on_test_data(model_tuned,X_test) #will return the predicted class
y_pred_prob = predict_prob_on_test_data(model_tuned,X_test) #model.predict_proba(X_test)
run_metrics = get_metrics(y_test, y_pred, y_pred_prob)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/media/indra-inc/ML_Doc/mflow_projects/mlflow-env/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/media/indra-inc/ML_Doc/mflow_projects/mlflow-env/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/media/indra-inc/ML_Doc/mflow_projects/mlflow-env/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/media/indra-inc/ML_Doc/mflow_projects/mlflow-env/lib/python3.10/site-packages/sklearn/b

[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=21; total time=   0.4s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=21; total time=   0.4s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=21; total time=   0.4s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=21; total time=   0.4s


/media/indra-inc/ML_Doc/mflow_projects/mlflow-env/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/media/indra-inc/ML_Doc/mflow_projects/mlflow-env/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/media/indra-inc/ML_Doc/mflow_projects/mlflow-env/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV] END bootstrap=True, max_depth=110, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=5; total time=   0.2s
[CV] END bootstrap=True, max_depth=110, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=5; total time=   0.2s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=21; total time=   0.3s
[CV] END bootstrap=True, max_depth=110, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=5; total time=   0.2s
[CV] END bootstrap=True, max_depth=110, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=5; total time=   0.2s


...

/media/indra-inc/ML_Doc/mflow_projects/mlflow-env/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=101; total time=   3.2s
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=101; total time=   3.2s
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=101; total time=   2.8s


/media/indra-inc/ML_Doc/mflow_projects/mlflow-env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
235 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
164 fits failed with the following error:
Traceback (most recent call last):
  File "/media/indra-inc/ML_Doc/mflow_projects/mlflow-env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/media/indra-inc/ML_Doc/mflow_projects/mlflow-env/lib/python3.10/site-packages/sklearn/base.py", line 1144, in wrapper
    estimator._validate_params()
  File "/media/indra-inc/ML_Doc/mflow_projects/mlflow-env/lib/python3.10/si

Random grid:  {'n_estimators': [5, 21, 51, 101], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120], 'min_samples_split': [2, 6, 10], 'min_samples_leaf': [1, 3, 4], 'bootstrap': [True, False]} 

Best Parameters:  {'n_estimators': 101, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 70, 'bootstrap': True}  



/media/indra-inc/ML_Doc/mflow_projects/mlflow-env/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [36]:
run_metrics 

{'accuracy': 0.91, 'precision': 0.67, 'recall': 0.42, 'entropy': 0.19}

In [37]:
for param in run_params:
    print(param, run_params[param])

n_estimators 101
min_samples_split 10
min_samples_leaf 1
max_features sqrt
max_depth 70
bootstrap True


In [38]:
create_experiment(experiment_name,run_name_tuned,model_name,run_metrics,model_tuned,run_params)

Experiment_id: 484267182419968048
Run Random_Search_CV_Tuned_Model with Run ID df3c15ae7b504e8797a1b244dfd82bde is logged to Experiment Banking Data Classification having status: RUNNING
